In [ ]:
# py 3.8.8
import pandas as pd
import os

In [ ]:
pwd = os.getcwd()

In [ ]:
dataset = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx",sheet_name="Edited_Data")
dataset

In [ ]:
dataset_modified = dataset.copy()
dataset_modified

In [ ]:
columns_to_drop = ['Start Date - ', 'End Date - ', 'Email Address - ','First Name - ', 'Last Name - ', 'Custom Data 1 - ']
columns_to_drop

In [ ]:
dataset_drop = dataset_modified.drop(columns=columns_to_drop)
dataset_drop

In [ ]:
id_vars = list(dataset_drop.columns)[0:8]
value_vars = list(dataset_drop.columns)[8 : ]
print(id_vars)
print(value_vars)

In [ ]:
'''pd.melt(id_vars=id_vars,value_vars=value_vars, ...) will unpivot columns of value_vars into rows, reducing the number of columns'''
dataset_melted = dataset_drop.melt(id_vars=id_vars,value_vars=value_vars, var_name="Question + Subquestion", value_name="Answer")
dataset_melted

In [ ]:
questions_import = pd.read_excel(pwd + "\\Data - Survey Monkey Output Edited.xlsx", sheet_name="Question")
questions_import

In [ ]:
questions = questions_import.copy()
questions.drop(columns=["Raw Question",	"Raw Subquestion", "Subquestion", 'Unnamed: 5'], inplace=True)
questions

In [ ]:
# df = pd.DataFrame([["Question 30", "Question 30 - Response 3"]], columns=["Question", "Question + Subquestion"])
# questions = questions.append(df, ignore_index=True)
# questions
'''pd.merge(how="left", ...) will match each row of the left dataset with the right dataset. Any duplicates on the right dataset will result in duplicated rows.'''
dataset_merged = pd.merge(left=dataset_melted, right=questions, how="left", left_on="Question + Subquestion", right_on="Question + Subquestion")
'''dataset_melted and dataset_merged len should be equal'''
print("Original Data", len(dataset_melted))
print("Merged Data", len(dataset_merged))
dataset_merged

In [ ]:
'''
this filter out rows of column "Answer" that have NaN.
'''
respondents = dataset_merged[dataset_merged["Answer"].notna()]
'''
pd.groupby(column_to_group) will group the dataset with the column_to_group
how many times a value occurs? the value are the rows of column_to_group.
for every row that contains the value 'Question 1', it will group the row to `Question 1`.
'''
respondents = respondents.groupby("Question")["Respondent ID - "].nunique().reset_index()
respondents.rename(columns={"Respondent ID - ":"Respondets"}, inplace=True)
respondents

In [ ]:
dataset_merged_two = pd.merge(left=dataset_merged, right=respondents, how="left", left_on="Question", right_on="Question")
print("Original Data", len(dataset_merged))
print("Merged Data", len(dataset_merged_two))
dataset_merged_two

In [ ]:
same_answer = dataset_merged#[dataset_merged["Answer"].notna()]
same_answer = same_answer.groupby(["Question + Subquestion", "Answer"])["Respondent ID - "].nunique().reset_index()
same_answer.rename(columns={"Respondent ID - ":"Same Answer"}, inplace=True)
same_answer

In [ ]:
dataset_merged_three = pd.merge(left=dataset_merged_two, right=same_answer, how="left", left_on=["Question + Subquestion", "Answer"], right_on=["Question + Subquestion", "Answer"])
dataset_merged_three["Same Answer"].fillna(0, inplace=True)
print("Original Data", len(dataset_merged_two))
print("Merged Data", len(dataset_merged_three))
dataset_merged_three

In [ ]:
output = dataset_merged_three.copy()
output.rename(columns={"Identify which division you work in. - Response":"Division Primary","Identify which division you work in. - Other (please specify)":"Division Secondary",	"Which of the following best describes your position level? - Response":"Position",	"Which generation are you apart of? - Response":"Generation", "Please select the gender in which you identify. - Response":"Gender",	"Which duration range best aligns with your tenure at your company? - Response":"Tenure",	"Which of the following best describes your employment type? - Response":"Employment Type"},inplace=True)
output

In [ ]:
output.to_excel(pwd + "\\Final_Output.xlsx", index=False)